# Mobile Website Generator

In [16]:
from selenium import webdriver
import pandas
import time
import os
import numpy as np
import signal

In [17]:
base = 430
bound = 499

In [18]:
websites_dataframe = pandas.read_csv('top-1m.csv', header=None)
website_urls = websites_dataframe.iloc[:,1].values
urls_to_process = website_urls[list(range(base, bound + 1))]

In [19]:
def save_website_image(driver, url, filename):
    driver.get(url)
    driver.save_screenshot(filename)

In [20]:
TIMEOUT_SEC = 20

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException
    
signal.signal(signal.SIGALRM, timeout_handler)

def save_website_images(driver, urls, dirname):
    url_counter = base
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    for url in urls:
        signal.alarm(TIMEOUT_SEC)
        
        try:
            driver.get('http://' + url)
            driver.save_screenshot(dirname + '/' + str(url_counter) + '.png')
            url_counter += 1
        except Exception:
            continue
        else:
            signal.alarm(0)
    return url_counter - base

In [21]:
def save_mobile_website_images(urls, dirname):
    mobile_emulation = { "deviceName": "Nexus 5" }
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
    driver = webdriver.Remote(command_executor='http://127.0.0.1:4444/wd/hub',
                              desired_capabilities = chrome_options.to_capabilities())
    
    return save_website_images(driver, urls, dirname)

In [22]:
start_time = time.time()
url_count = save_mobile_website_images(urls_to_process, 'random_sites_mobile')
elapsed_time = time.time() - start_time
print('Elapsed time:', elapsed_time)
print('Number of URLs saved:', url_count)
print('Average time per URL:', elapsed_time / url_count)

Elapsed time: 791.7857728004456
Number of URLs saved: 39
Average time per URL: 20.302199302575527
